In [36]:
import nibabel as nib
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from tensorflow.keras import layers, models, callbacks, metrics, Input, Model, regularizers
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
import cv2
import random

In [37]:
# Definir nível de log para erro (ignora warnings e info)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

In [38]:
def calculate_label(image, threshold=0.01):
    """
    Determina o label da subimagem com base no percentual de fundo não-preto.
    :param subimage: Array da subimagem.
    :param threshold: Percentual mínimo de fundo não-preto para considerar como label 1.
    :return: String indicando o label.
    """
    # Total de pixels na subimagem
    total_pixels = image.size
    # Número de pixels não-preto
    non_zero_pixels = np.count_nonzero(image)
    # Proporção de pixels não-preto
    non_black_ratio = non_zero_pixels / total_pixels if total_pixels > 0 else 0
    
    # Verifica se há lesão e se o fundo não-preto é maior que o limiar
    if np.any(image == 1) and non_black_ratio >= threshold:
        return 1
    else:
        return 0

def load_patient_data(folder, patient_id):
    """
    Carrega os dados de um único paciente (imagens, máscaras e labels) de um diretório.

    Args:
        folder (str): Caminho da pasta contendo os dados dos pacientes.
        patient_id (str): ID do paciente a ser carregado.

    Returns:
        dict: Dados do paciente, incluindo imagens, máscaras e labels para os lados esquerdo e direito.
              Retorna None se o paciente não for encontrado.
    """
    patient_path = os.path.join(folder, patient_id)
    if not os.path.exists(patient_path):
        print(f"Paciente {patient_id} não encontrado na pasta {folder}.")
        return None

    # Inicializa estruturas para armazenar os dados do paciente
    patient_data = {
        "images_left": [],
        "images_right": [],
        "mask_left": [],
        "mask_right": [],
        "labels_left": [],
        "labels_right": [],
    }

    areas_image = ["left", "right"]
    areas_mask = ["lesion_left", "lesion_right"]
    path_left = os.path.join(patient_path, areas_image[0])
    path_right = os.path.join(patient_path, areas_image[1])
    lesion_path_left = os.path.join(patient_path, areas_mask[0])
    lesion_path_right = os.path.join(patient_path, areas_mask[1])

    # Verifica se os diretórios existem
    if not os.path.exists(path_left) or not os.path.exists(path_right) or \
       not os.path.exists(lesion_path_left) or not os.path.exists(lesion_path_right):
        print(f"Estrutura de diretórios inválida para o paciente {patient_id}.")
        return None

    # Carrega as imagens e máscaras do lado esquerdo e direito
    for patch_id_left, mask_id_left, patch_id_right, mask_id_right in zip(
        os.listdir(path_left), os.listdir(lesion_path_left),
        os.listdir(path_right), os.listdir(lesion_path_right)
    ):
        img_path_left = os.path.join(path_left, patch_id_left)
        mask_path_left = os.path.join(lesion_path_left, mask_id_left)
        img_path_right = os.path.join(path_right, patch_id_right)
        mask_path_right = os.path.join(lesion_path_right, mask_id_right)

        for img_left, msk_left, img_right, msk_right in zip(
            os.listdir(img_path_left), os.listdir(mask_path_left),
            os.listdir(img_path_right), os.listdir(mask_path_right)
        ):
            # Carrega os dados do lado esquerdo
            data_left = nib.load(os.path.join(img_path_left, img_left)).get_fdata()
            data_msk_left = nib.load(os.path.join(mask_path_left, msk_left)).get_fdata()
            if len(data_left) > 0 or len(data_msk_left) > 0:
                patient_data["images_left"].append(data_left)
                patient_data["mask_left"].append(data_msk_left)
                patient_data["labels_left"].append(calculate_label(data_msk_left))

            # Carrega os dados do lado direito
            data_right = nib.load(os.path.join(img_path_right, img_right)).get_fdata()
            data_msk_right = nib.load(os.path.join(mask_path_right, msk_right)).get_fdata()
            if len(data_right) > 0 or len(data_msk_right) > 0:
                patient_data["images_right"].append(data_right)
                patient_data["mask_right"].append(data_msk_right)
                patient_data["labels_right"].append(calculate_label(data_msk_right))

    # Gera os pares de labels
    labels_pair = []
    for label_left, label_right in zip(patient_data["labels_left"], patient_data["labels_right"]):
        if label_left == 0 and label_right == 0:
            labels_pair.append(0)
        else:
            labels_pair.append(1)
    patient_data["labels_pair"] = labels_pair

    print(f"Paciente {patient_id} carregado com sucesso.")
    print(f"Total de recortes: {len(labels_pair)}")
    return patient_data, labels_pair

In [39]:
# Função para normalizar entre 0 e 1
def normalize_minmax(image_data): 
    min_val = np.min(image_data)
    max_val = np.max(image_data)
    normalized_data = (image_data - min_val) / (max_val - min_val)
    return normalized_data

# Função para filtrar as imagens por paciente
def select_by_patients(patients, all_images_original, all_images_opposite, all_labels):
    selected_images_original = {}
    selected_images_opposite = {}
    selected_labels = []
    
    for patient in patients:
        selected_images_original[patient] = []
        selected_images_opposite[patient] = []
        if patient in all_images_original and patient in all_images_opposite:
            selected_images_original[patient].extend(all_images_original[patient])
            selected_images_opposite[patient].extend(all_images_opposite[patient])
            selected_labels.extend(all_labels[patient])
        else:
            print(f"Paciente {patient} não encontrado em uma das listas de imagens.")
    
    return selected_images_original, selected_images_opposite, selected_labels

In [40]:
# Função para preparar os dados para treino e validação
def prepare_data_for_training(images_left, images_right, labels_pair, mask_left, mask_right, train_size=0.7, validation_size=0.2, test_size=0.1, augment_factor=1):
    balanced_images_left = {}
    balanced_images_right = {}
    balanced_labels = {}
    balanced_mask_left = {}
    balanced_mask_right = {}
    
    for patient_id in images_left:
        class_1_labels = []
        class_0_labels = []
        class_1_left = {}
        class_0_left = {}
        class_1_right = {}
        class_0_right = {}
        class_1_mask_left = {}
        class_0_mask_left = {}
        class_1_mask_right = {}
        class_0_mask_right = {}
        class_1_coordinates = {}
        class_0_coordinates = {}
        class_1_left[patient_id] = []
        class_0_left[patient_id] = []
        class_1_right[patient_id] = []
        class_0_right[patient_id] = []
        class_1_mask_left[patient_id] = []
        class_0_mask_left[patient_id] = []
        class_1_mask_right[patient_id] = []
        class_0_mask_right[patient_id] = []
        class_1_coordinates[patient_id] = []
        class_0_coordinates[patient_id] = []
        index = 0

        train_patients = ['sub-00H10', 'sub-06C09', 'sub-14F04', 'sub-16E03', 'sub-16I12', 'sub-19G04', 'sub-22F14', 'sub-25B08', 'sub-26B09', 'sub-34J06', 'sub-35E12', 'sub-36K02', 'sub-42K06', 'sub-03C08', 'sub-51C05', 'sub-52K04', 'sub-56E13', 'sub-59E09''sub-41D08', 'sub-57D04', 'sub-59G00', 'sub-60G06', 'sub-72K02', 'sub-76J09', 'sub-79H07', 'sub-83K08', 'sub-85I05', 'sub-86B13', 'sub-86G08']
        valid_patients = ['sub-02A13', 'sub-16G09', 'sub-29D03', 'sub-31F07', 'sub-19F09', 'sub-42B05', 'sub-44H05', 'sub-76E02']
        test_patients = ['sub-03C08', 'sub-51C05', 'sub-52K04', 'sub-59E09']

        # Iterar sobre os patches e dividir as classes 0 e 1 com base nas labels\n",
        if len(labels_pair[patient_id]) == 0:
            continue
        
        for label in labels_pair[patient_id]:
            image_left = (images_left[patient_id])[index]
            image_right = (images_right[patient_id])[index]
            mask_image_left = (mask_left[patient_id])[index]
            mask_image_right = (mask_right[patient_id])[index]
            index += 1
            
            if label == 1:
                class_1_labels.append(label)
                class_1_left[patient_id].append(image_left)
                class_1_right[patient_id].append(image_right)
                class_1_mask_left[patient_id].append(mask_image_left)
                class_1_mask_right[patient_id].append(mask_image_right)
            else:
                class_0_labels.append(label)
                class_0_left[patient_id].append(image_left)
                class_0_right[patient_id].append(image_right)
                class_0_mask_left[patient_id].append(mask_image_left)
                class_0_mask_right[patient_id].append(mask_image_right)
        class_1_count = len(class_1_labels)
        print(f"Paciente {patient_id}: Total de pares de recortes com labels 1: {class_1_count}")

        # Fazer undersampling aleatório da classe majoritária (label 0) para igualar ao número de exemplos da classe 1
        class_0_count = len(class_0_labels)
        if class_0_count > class_1_count and  patient_id not in test_patients:
            sampled_indices = [i for i in range(class_1_count)]
            sampled_class_0_slices_left = [class_0_left[patient_id][i] for i in sampled_indices]
            sampled_class_0_slices_right = [class_0_right[patient_id][i] for i in sampled_indices]
            class_0_original_modified = {}
            class_0_original_modified[patient_id] = []
            class_0_opposite_modified = {}
            class_0_opposite_modified[patient_id] = []
            sampled_class_0_mask_left = [class_0_mask_left[patient_id][i] for i in sampled_indices]
            sampled_class_0_mask_right = [class_0_mask_right[patient_id][i] for i in sampled_indices]
            class_0_mask_original_modified = {}
            class_0_mask_original_modified[patient_id] = []
            class_0_mask_opposite_modified = {}
            class_0_mask_opposite_modified[patient_id] = []

            for left_image, left_mask in zip(sampled_class_0_slices_left, sampled_class_0_mask_left):
                class_0_original_modified[patient_id].append(left_image)
                class_0_mask_original_modified[patient_id].append(left_mask)
            for right_image, right_mask in zip(sampled_class_0_slices_right, sampled_class_0_mask_right):
                class_0_opposite_modified[patient_id].append(right_image)
                class_0_mask_opposite_modified[patient_id].append(right_mask)
            class_0_labels = [0] * class_1_count

        if patient_id in test_patients:
            class_0_original_modified = class_0_left
            class_0_opposite_modified = class_0_right
            class_0_mask_original_modified = class_0_mask_left
            class_0_mask_opposite_modified = class_0_mask_right

        balanced_images_left[patient_id] = list(class_1_left[patient_id]) + list(class_0_original_modified[patient_id])
        balanced_images_right[patient_id] = list(class_1_right[patient_id]) + list(class_0_opposite_modified[patient_id])
        balanced_labels[patient_id] = class_1_labels + class_0_labels
        balanced_mask_left[patient_id] = list(class_1_mask_left[patient_id]) + list(class_0_mask_original_modified[patient_id])
        balanced_mask_right[patient_id] = list(class_1_mask_right[patient_id]) + list(class_0_mask_opposite_modified[patient_id])
        
        balanced_index = np.random.choice(range(len(balanced_labels[patient_id])), len(balanced_labels[patient_id]), replace=False)            
        balanced_images_left[patient_id] = [balanced_images_left[patient_id][i] for i in balanced_index]
        balanced_images_right[patient_id] = [balanced_images_right[patient_id][i] for i in balanced_index]
        balanced_labels[patient_id] = [balanced_labels[patient_id][i] for i in balanced_index]
        balanced_mask_left[patient_id] = [balanced_mask_left[patient_id][i] for i in balanced_index]
        balanced_mask_right[patient_id] = [balanced_mask_right[patient_id][i] for i in balanced_index]
        
    # Separar os dados por conjunto (treino, validação, teste)
    #patients = list(set(balanced_labels.keys()))
    #train_patients, valtest_patients = train_test_split(patients, train_size = train_size)
    #valid_patients, test_patients = train_test_split(valtest_patients, train_size = validation_size / (validation_size + test_size))


    X_train_original, X_train_opposite, y_train = select_by_patients(train_patients, balanced_images_left, balanced_images_right, balanced_labels)
    X_val_original, X_val_opposite, y_val = select_by_patients(valid_patients, balanced_images_left, balanced_images_right, balanced_labels)
    X_test_original, X_test_opposite, y_test = select_by_patients(test_patients, balanced_images_left, balanced_images_right, balanced_labels)
    
    print(f"Total de pares de recortes no treino ({augment_factor}*{sorted(train_patients)}) com label 1: {y_train.count(1)}")
    print(f"Total de pares de recortes no treino ({augment_factor}*{sorted(train_patients)}) com label 0: {y_train.count(0)}")
    print(f"Total de pares de recortes na validação ({augment_factor}*{sorted(valid_patients)}) com label 1: {y_val.count(1)}")
    print(f"Total de pares de recortes na validação ({augment_factor}*{sorted(valid_patients)}) com label 0: {y_val.count(0)}")
    print(f"Total de pares de recortes no teste com ({augment_factor}*{sorted(test_patients)}) label 1: {y_test.count(1)}")
    print(f"Total de pares de recortes no teste com ({augment_factor}*{sorted(test_patients)}) label 0: {y_test.count(0)}")
    
    return X_train_original, X_train_opposite, X_val_original, X_val_opposite, X_test_original, X_test_opposite, np.array(y_train), np.array(y_val), np.array(y_test), train_patients, valid_patients, test_patients, balanced_index

In [41]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def salvar_pares_imagens_pdf(left_images, right_images, pdf_filename):
    """
    Salva pares de imagens em um PDF, com 4 pares (8 imagens) por página.

    Parâmetros:
    - left_images: Lista ou array com as imagens do lado esquerdo.
    - right_images: Lista ou array com as imagens do lado direito.
    - pdf_filename: Nome do arquivo PDF de saída.

    """
    with PdfPages(pdf_filename) as pdf:
        image_pairs = list(zip(left_images, right_images))
        
        for i in range(0, len(image_pairs), 4):  # Processando 4 pares por página
            fig, axes = plt.subplots(4, 2, figsize=(10, 15))  # Criando grade 4x2
            
            for j in range(4):  # Adicionando 4 pares por página
                if i + j < len(image_pairs):  # Verifica se ainda há imagens disponíveis
                    l_image, r_image = image_pairs[i + j]
                    
                    # Primeira coluna (Recorte Esquerda)
                    axes[j, 0].imshow(l_image, cmap='gray')
                    axes[j, 0].axis('off')
                    axes[j, 0].set_title(f"Recorte Esquerda {i+j+1}")
                    
                    # Segunda coluna (Recorte Direita)
                    axes[j, 1].imshow(r_image, cmap='gray')
                    axes[j, 1].axis('off')
                    axes[j, 1].set_title(f"Recorte Direita {i+j+1}")

            # Ajustar layout e salvar no PDF
            plt.tight_layout()
            pdf.savefig(fig)  
            plt.close(fig)

    print(f"PDF salvo como {pdf_filename}")

In [42]:
# Caminho da pasta contendo os dados dos pacientes
folder = "Novo_Contralateral"

# Lista de IDs dos pacientes
patient_ids = [os.listdir(folder)[0]]
#patient = os.listdir(folder)[0]

X_left, X_right, y, mask_left, mask_right = {}, {}, {}, {}, {}

# AJUSTE PRA TER NO DICIONÁRIO CADA NÍVEL DE UM PACIENTE

# Processa um paciente por vez
for patient_id in patient_ids:
    # Carrega os dados do paciente
    patient_data, labels_pair = load_patient_data(folder, patient_id)
    
    if patient_data is not None:
        X_left[patient_id] = patient_data["images_left"]
        X_right[patient_id] = patient_data["images_right"]
        mask_left[patient_id] = patient_data["mask_left"]
        mask_right[patient_id] = patient_data["mask_right"]
        y[patient_id] = labels_pair

Paciente sub-00H10 carregado com sucesso.
Total de recortes: 1026


In [43]:
print(patient_ids)
print(len(X_left['sub-00H10']))
print(len(X_right['sub-00H10']))
print(len(mask_left['sub-00H10']))
print(len(mask_right['sub-00H10']))
print(len(y['sub-00H10']))
print(y['sub-00H10'].count(0))
print(y['sub-00H10'].count(1))

['sub-00H10']
1026
1026
1026
1026
1026
966
60


In [44]:
# Nome do arquivo PDF
os.makedirs("Pdf", exist_ok=True)
pdf_filename = "Pdf/imagens_recortes_pre_undersampling.pdf"

salvar_pares_imagens_pdf(X_left['sub-00H10'], X_right['sub-00H10'], pdf_filename)

PDF salvo como Pdf/imagens_recortes_pre_undersampling.pdf


In [45]:
# Preparar dados para treino e validação
train_left_balanced, train_right_balanced, valid_left_balanced, valid_right_balanced, test_left, test_right, y_train_balanced, y_valid_balanced, y_test, train_patients, valid_patients, test_patients, index_vec = prepare_data_for_training(X_left, X_right, y, mask_left, mask_right, train_size=0.7, validation_size=0.2, test_size=0.1)

# Adiciono camada de cor
train_left_balanced = normalize_minmax(np.array([elemento for lista in train_left_balanced.values() for elemento in lista]))
train_left_balanced = np.expand_dims(train_left_balanced, axis=-1)
train_right_balanced = normalize_minmax(np.array([elemento for lista in train_right_balanced.values() for elemento in lista]))
train_right_balanced = np.expand_dims(train_right_balanced, axis=-1)

Paciente sub-00H10: Total de pares de recortes com labels 1: 60
Paciente sub-06C09 não encontrado em uma das listas de imagens.
Paciente sub-14F04 não encontrado em uma das listas de imagens.
Paciente sub-16E03 não encontrado em uma das listas de imagens.
Paciente sub-16I12 não encontrado em uma das listas de imagens.
Paciente sub-19G04 não encontrado em uma das listas de imagens.
Paciente sub-22F14 não encontrado em uma das listas de imagens.
Paciente sub-25B08 não encontrado em uma das listas de imagens.
Paciente sub-26B09 não encontrado em uma das listas de imagens.
Paciente sub-34J06 não encontrado em uma das listas de imagens.
Paciente sub-35E12 não encontrado em uma das listas de imagens.
Paciente sub-36K02 não encontrado em uma das listas de imagens.
Paciente sub-42K06 não encontrado em uma das listas de imagens.
Paciente sub-03C08 não encontrado em uma das listas de imagens.
Paciente sub-51C05 não encontrado em uma das listas de imagens.
Paciente sub-52K04 não encontrado em uma

In [46]:
#train_left_balanced, train_right_balanced, train_patients

print(len(train_left_balanced))
print(len(train_right_balanced))
print(len(y_train_balanced))

120
120
120


In [47]:
# Nome do arquivo PDF
os.makedirs("Pdf", exist_ok=True)
pdf_filename = "Pdf/imagens_recortes_pos_undersampling.pdf"

salvar_pares_imagens_pdf(train_left_balanced, train_right_balanced, pdf_filename)

PDF salvo como Pdf/imagens_recortes_pos_undersampling.pdf


In [48]:
valid_left_balanced = normalize_minmax(np.array([elemento for lista in valid_left_balanced.values() for elemento in lista]))
valid_left_balanced = np.expand_dims(valid_left_balanced, axis=-1)
valid_right_balanced = normalize_minmax(np.array([elemento for lista in valid_right_balanced.values() for elemento in lista]))
valid_right_balanced = np.expand_dims(valid_right_balanced, axis=-1)
test_left = normalize_minmax(np.array([elemento for lista in test_left.values() for elemento in lista]))
test_left = np.expand_dims(test_left, axis=-1)
test_right = normalize_minmax(np.array([elemento for lista in test_right.values() for elemento in lista]))
test_right = np.expand_dims(test_right, axis=-1)

ValueError: zero-size array to reduction operation minimum which has no identity